In [1]:
import numpy as np
np.__version__

'2.0.2'

# Part 1

Need to find max joltage across all banks of batteries.

### Data Import and Exploration

In [2]:
# Import raw data as numpy array
data = np.loadtxt(
    fname='Input3.txt',
    dtype=str  # very important! string preferred over float, to keep data intact
)

In [3]:
data[:5]

array(['2412122322321222252222221622332222221521261431723112634211232223232132154222214222211332229222231222',
       '2465412223443523142221321262346323634242443342334242423242224362522642253213243122355223222254312232',
       '3332242442321436631424453283363222444423355334373183244397472932323244521266474444834444476564344644',
       '5533778667855399367659688956396486549458366658763989334992855699859653338632354754294593558359588565',
       '2312242244513323142433243432334374124436822261433652333532361223332463425141733353743352723352424632'],
      dtype='<U100')

Data is a matrix of integers, with each row representing a bank.  
Plan of attack: capture data in *n* x *m* matrix

In [4]:
print(f"number of rows, n:\t{len(data)}")

number of rows, n:	200


### Helpers

In [5]:
def digit_count(input_int: int) -> int:
    """For a given integer input, return number of digits."""
    # instantiate counter
    digit_counter = 0

    # increment digit counter using modulo function
    while True:
        if (input_int % (10**digit_counter)) == input_int:
            return digit_counter
        else:
            digit_counter += 1
        

In [6]:
def retrieve_digits(number: int, position: int, count: int = 1) -> int:
    """Returns digit(s) of integer from given position, 
    
    1-indexed from units upwards.
    """
    # bound the position argument
    if (position > digit_count(number)) | (position < 1):
        return 0
    
    # modulo function logic for retrival of digits
    return int(
        ((number % 10**position) - (number % 10**(position-count))) / 10**(position-count)
    )

In [7]:
print(f"unique column counts:\t{set([digit_count(int(i)) for i in data])}")

unique column counts:	{100}


200 x 100 matrix should do it...

In [8]:
# instantiate matrix of battery banks
bank_matrix = np.zeros(
    shape=(200, 100),
    dtype=int
)

# populate matrix
for i, j in enumerate(data):
    for k in range(digit_count(int(j))):  # k is iterator over bank length
        bank_matrix[i, k] = retrieve_digits(int(j), digit_count(int(j)) - k)

In [9]:
# test matrix completeness versus raw data
_data_integrity = []
for i, j in enumerate(data):
    for k in range(len(j)):
        # is each number in raw data equal to corresponding element in matrix?
        _data_integrity.append(int(j[k]) == bank_matrix[i, k])

print("OK!") if np.all(_data_integrity) else print("CHECK!")

OK!


In [10]:
data[0]

np.str_('2412122322321222252222221622332222221521261431723112634211232223232132154222214222211332229222231222')

In [11]:
bank_matrix[0]

array([2, 4, 1, 2, 1, 2, 2, 3, 2, 2, 3, 2, 1, 2, 2, 2, 2, 5, 2, 2, 2, 2,
       2, 2, 1, 6, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 1, 5, 2, 1, 2, 6, 1, 4,
       3, 1, 7, 2, 3, 1, 1, 2, 6, 3, 4, 2, 1, 1, 2, 3, 2, 2, 2, 3, 2, 3,
       2, 1, 3, 2, 1, 5, 4, 2, 2, 2, 2, 1, 4, 2, 2, 2, 2, 1, 1, 3, 3, 2,
       2, 2, 9, 2, 2, 2, 2, 3, 1, 2, 2, 2])

### Max Joltage

For each bank of 2 selected batteries, find max joltage.

### More Helpers

In [12]:
def new_argmax(input_array: np.array, *max_value: int) -> list:
    """Function that returns every index of max values in an array.
    
    Works for 1d arrays.
    Returns tuple: (max value, array of indices of occurence)
    """
    
    # check for null array
    if not any(input_array):
        return (0, np.array([]))
    
    # instantiate list of max value indices
    output_array = []
    # max value, if supplied or found in the array
    val = max_value if any(max_value) else (max(input_array),)

    return val[0], np.where(input_array == val[0])[0]

In [13]:
def stitch(numbers: list) -> int:
    """Given a list of single integers, stitch back to one integer.
    
    Be aware of lists that are too long; limited by the length of long type in C
    """
    # output integer
    output = 0
    
    # error check argument
    if (len(numbers) < 1):
        return 0

    # stitch together
    for i in range(len(numbers), 0, -1):
        output += (10**(i-1)) * numbers[len(numbers) - i]

    return int(output)

In [14]:
def max_joltage_2_batteries(input_array: np.array) -> int:
    """Ouput max joltage for a given input array.
    
    Logic is for two batteries.
    """
    
    # select 2 batteries
    batteries = 2
    
    # check for null array
    if not any(input_array):
        return 0
    
    # instantiate array (before stitching) of max joltage
    output_joltage = np.zeros(
        shape=(batteries,),
        dtype=int
    )
    # initial max value
    max_value = new_argmax(input_array)[0]
    
    # indices of initial max value occurences
    indices = new_argmax(input_array)[1]

    # working variable for list of array splits
    splits = []

    # working variable for remaining batteries to select
    remaining_batteries = max(0, batteries - len(indices))

    # ===== AT LEAST 2 MAX VALUES =====

    # if there are more max value occurences than batteries needed...
    if not remaining_batteries:
        # ...then populate digits for joltage
        for i in range(batteries):
            output_joltage[i] = input_array[indices[i]]
            
        return stitch(output_joltage)
            
    # ===== ONE MAX VALUE =====
    
    # split array to see position context of batteries
    splits = np.split(input_array, indices)
    
    # edge case 1: the max value the last in the array
    if len(splits[-1]) == 1:
        for i in range(batteries):
            output_joltage[i] = new_argmax(splits[i])[0]  # populate digits for joltage
            
    # edge case 2: the max value the first in the array
    elif len(splits[0]) == 0:
        for i in range(batteries):
            if i == 0:
                output_joltage[i] = splits[1][i]  # populate first digit of joltage
                
            else:
                output_joltage[i] = new_argmax(splits[i][1:])[0]  # populate second digit for joltage
                
    # all other positions of max value 
    else:
        for i in range(batteries):
            if i == 0:
                output_joltage[i] = splits[1][i]  # populate first digit of joltage
                
            else:
                output_joltage[i] = new_argmax(splits[i][1:])[0]  # populate second digit for joltage
    
    return stitch(output_joltage)

### Total max joltage

In [15]:
res = sum([max_joltage_2_batteries(i) for i in bank_matrix])
res

16973

# Part 2

As initially suspected, need to find max joltage per bank; this time for 12 batteries per bank.

In [16]:
def max_joltage_n_batteries(input_array: np.array, batteries: int = 12) -> int:
    """Ouput max joltage for a given input array.
    
    Default is for twelve batteries, but can be up to number of batteries in the bank.
    """
    
    # error checker
    if batteries >= len(input_array):
        return stitch(input_array)
    
    # instantiate array (before stitching) of max joltage
    output_joltage = np.zeros(
        shape=(batteries,),
        dtype=int
    )

    # initial max value
    max_value_init = new_argmax(input_array)[0]
    
    # indices of initial max value occurences
    indices = new_argmax(input_array)[1]

    # initial list of array splits
    splits_init = []

    # working variable for remaining batteries to calculate (apart from max values)
    remaining_batteries = max(0, batteries - len(indices))
    
    # counter for batteries added
    batteries_added = 0
    
    # working variable for a 'sub-joltage' in recursion of sub-splits
    sub_joltage = 0
    
    
    # ===== MORE MAX VALUES THAN BATTERIES =====

    # if there are more max value occurences than batteries needed...
    if not remaining_batteries:
        # ...then populate digits for joltage
        for i in range(batteries):
            output_joltage[i] = input_array[indices[i]]
        
        return stitch(output_joltage)
    
    
    # ===== LESS MAX VALUES THAN BATTERIES =====

    # split array to see position context of batteries
    # splits by each max value, which is placed at start of sub-array. there are always len(indices) + 1 splits, even if a split is empty.
    splits_init = np.split(input_array, indices)

    # aside from max values, the remaining batteries need to be calculated for the maximum joltage.
    # PLAN OF ATTACK: iterate from right-most split, filling up calculated digits until remaining_batteries == 0
    # and also fill the max values
    
    for i in range(1, len(splits_init) + 1):  # i is from 1 to len(splits_init)
        
        # skip over 0 length splits -> go to next split
        if not len(splits_init[-i]):
            continue
        
        
        # ===== case 1 =====
        # no more remaining batteries to calculate. populate with the initial max value (first in split)
        
        if not remaining_batteries:

            # terminate insertion at end of output array
            if (batteries_added + 1) <= len(output_joltage):
                # add split max value to output
                if i <= len(indices):
                    output_joltage[-(batteries_added+1)] = input_array[indices[-i]]

            # update added battery after update
            batteries_added += 1
            
            continue
        
        
        # ===== case 2 =====
        # need to use all of split digits (excluding max value) for joltage
        
        if len(splits_init[-i]) <= remaining_batteries:  # start from rightmost split
            
            # decrement remaining batteries to be calculated
            remaining_batteries -= len(splits_init[-i][1:])
            
            # populate output with whole split from rhs
            for j in range(1, len(splits_init[-i]) + 1):  # j is from 1 to len(sub-split)
                
                # terminate insertion at end of output array
                if (j + batteries_added) <= len(output_joltage):
                    output_joltage[-(j+batteries_added)] = splits_init[-i][-j]
                
            # update added batteries after full update
            batteries_added += len(splits_init[-i])
            
            
        # ===== case 3 =====
        # more split digits than batteries outstanding. Recursively find maximal sub-joltage.
        
        elif len(splits_init[-i]) > remaining_batteries:
            
            # recursive search for max joltage for split
            
            # first element of this split is a max value (i.e. will be included in joltage), \
            # then find max joltage of other elements
            if max_value_init in splits_init[-i]:
                sub_joltage = max_joltage_n_batteries(splits_init[-i][1:], remaining_batteries)
            
            # otherwise find max joltage of entire split
            else:
                sub_joltage = max_joltage_n_batteries(splits_init[-i], remaining_batteries)
            
            # add split sub-joltage digits to output
            for k in range(1, remaining_batteries+1):  # k is from 1 to remaining batteries
                output_joltage[-batteries_added-k] = retrieve_digits(sub_joltage, k)
            
            # update added batteries after full update of sub-joltage
            batteries_added += digit_count(sub_joltage) 
            
            # add split max value to output (if applicable)
            if i <= len(indices):
                if splits_init[-i][0] == input_array[indices[-i]]:
                    if (batteries_added + 1) <= len(output_joltage):  # terminate insertion at end of output array
                        output_joltage[-(batteries_added+1)] = splits_init[-i][0]

                    # update added batteries after update of max value
                    batteries_added += 1
            
            # no more batteries to be chosen
            remaining_batteries = 0

    return stitch(output_joltage)

In [17]:
sum([max_joltage_n_batteries(i, 12) for i in bank_matrix])

168027167146027